# Computer vision template
This follows https://auto.gluon.ai/stable/tutorials/multimodal/image_prediction/beginner_image_cls.html

In [ ]:
import os
# Use the third GPU onlu
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path

# Load images from the images directory
df = pd.DataFrame([str(i) for i in Path('images').glob('*.tif')], columns=['path'])

# Load the property that you want to predict
df_out = pd.read_csv('data_to_predict')

# Merge both to data set
df = pd.concat((df, df_out), axis=1)

# Split
df_train, df_test = train_test_split(df, test_size=0.20, random_state=42)

In [ ]:
from autogluon.multimodal import MultiModalPredictor
import uuid
model_path = f"./tmp/{uuid.uuid4().hex}-automm_shopee"
predictor = MultiModalPredictor(label="area", path=model_path)
predictor.fit(
    train_data=df_train,
    time_limit=50, # seconds
    hyperparameters={"env.num_gpus": 1, 'optimization.max_epochs': 100},
)

In [ ]:
import matplotlib.pyplot as plt
pred = predictor.predict(df_test)
df_test.area

plt.plot(df_test.area, pred, 'o')
plt.plot([10, 60], [10, 60], 'k--')
